# NOMAD Authentication Module

This notebook provides reusable authentication UI and logic for NOMAD API access. It can be imported into other dashboard notebooks.

## Usage

```python
# Import the authentication module
%run 'nomad_auth.ipynb'

# The following variables are now available:
# - current_token: The authenticated token
# - current_user_info: Information about the authenticated user
# - api_client: NomadClient instance for making API calls
# - general_settings_box: Widget containing the authentication UI
```

In [7]:
# Import necessary libraries
import ipywidgets as widgets
from IPython.display import display, clear_output
import os   # To read environment variables
import json # To handle JSON responses

# Import the NOMAD API module
from nomad_api.auth import OASIS_OPTIONS, authenticate, get_token, get_token_from_env, get_credentials_from_env, verify_token
from nomad_api.client import NomadClient

# --- Define Oasis Options ---
# Use predefined options from the nomad_api.auth module
oasis_options = OASIS_OPTIONS

# --- Application State ---
# Store the currently active token after successful authentication
current_token = None
# Store the authenticated user info (optional, but useful)
current_user_info = None
# Store the API client instance after authentication
api_client = None

# Create a configuration module to store authentication state
class NomadAuthConfig:
    # --- Application State ---
    # Store the currently active token after successful authentication
    current_token = None
    # Store the authenticated user info (optional, but useful)
    current_user_info = None
    # Store the API client instance after authentication
    api_client = None

# Instance of auth config that will be used by this module
auth_config = NomadAuthConfig()

In [ ]:
# --- General Settings Widgets ---

# Dropdown for selecting NOMAD Oasis by name
# Ensure there's at least one option, otherwise widget creation might fail
if not oasis_options:
    oasis_options["Default (Please Edit Code)"] = "http://example.com/api/v1"

oasis_dropdown = widgets.Dropdown(
    options=oasis_options.keys(),
    value=list(oasis_options.keys())[0], # Default to the first option
    description='Select Oasis:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(min_width='250px', flex='1 1 auto')
)

# Label to display the URL of the selected Oasis (read-only)
selected_url_display = widgets.Label(
    # Handle case where oasis_options might be initially empty
    value=f"Base URL: {oasis_options.get(oasis_dropdown.value, 'N/A')}",
    layout=widgets.Layout(margin='5px 0 5px 5px', width='auto') # Added bottom margin
)

# --- Authentication Widgets ---

# Radio buttons to select authentication method
auth_method_selector = widgets.RadioButtons(
    options=['Username/Password', 'Token (from ENV)'], # Updated label for clarity
    description='Auth Method:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(margin='10px 0 0 0')
)

# --- Username/Password Widgets ---
username_input = widgets.Text(
    placeholder='Enter Username (e.g., email) or leave empty to use from ENV',
    description='Username:',
    style={'description_width': 'initial'}
)
password_input = widgets.Password(
    placeholder='Enter Password or leave empty to use from ENV',
    description='Password:',
    style={'description_width': 'initial'}
)
# Box to hold U/P inputs, initially hidden/shown based on environment
local_auth_box = widgets.VBox([username_input, password_input], layout=widgets.Layout(margin='5px 0 0 0'))

# --- Token Widget ---
token_input = widgets.Password(
    placeholder='Token will be read from ENV', # Updated placeholder
    description='Token:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='95%'),
    disabled=True # Disable input as it's not used directly
)
# Box to hold token input, initially hidden/shown based on environment
token_auth_box = widgets.VBox([token_input], layout=widgets.Layout(margin='5px 0 0 0', display='none'))

# --- Authentication Action Widgets ---
auth_button = widgets.Button(
    description='Authenticate',
    button_style='info',
    tooltip='Authenticate using the selected method',
    icon='unlock',
    layout=widgets.Layout(width='120px', margin='10px 0 0 0') # Set fixed width of 120px
)

auth_status_label = widgets.Label(
    value='Status: Not Authenticated',
    layout=widgets.Layout(margin='5px 0 0 0')
)

# Box to hold the button and status
auth_action_box = widgets.VBox([auth_button, auth_status_label])

# --- Arrange General Settings Widgets ---
# Combine all authentication widgets vertically
auth_box = widgets.VBox([
    auth_method_selector,
    local_auth_box,  # Will be shown/hidden by observer
    token_auth_box,  # Will be hidden by observer when Token method selected
    auth_action_box
])

# Main settings box layout
general_settings_box = widgets.VBox([
    widgets.HTML("<h2>General Settings</h2>"),
    oasis_dropdown,
    selected_url_display,
    auth_box # Add the combined authentication section
], layout=widgets.Layout(border='1px solid #ccc', padding='10px', margin='0 0 20px 0'))

NameError: name 'oasis_options' is not defined

In [9]:
# --- Observers and Handler Functions ---

# Observer for Oasis change
def on_oasis_change(change):
    global current_token, current_user_info, api_client # Allow modification of global state
    if change['type'] == 'change' and change['name'] == 'value':
        selected_name = change['new']
        selected_url = oasis_options.get(selected_name, "Invalid selection - URL not found")
        selected_url_display.value = f"Base URL: {selected_url}"
        # Reset authentication status and token when Oasis changes
        auth_status_label.value = 'Status: Not Authenticated (Oasis changed)'
        auth_status_label.style.text_color = None
        # Reset auth config state
        auth_config.current_token = None
        auth_config.current_user_info = None
        auth_config.api_client = None
        # Update global variables
        current_token = None
        current_user_info = None
        api_client = None

oasis_dropdown.observe(on_oasis_change, names='value')

# Observer for Auth Method change
def on_auth_method_change(change):
    global current_token, current_user_info, api_client # Allow modification of global state
    if change['type'] == 'change' and change['name'] == 'value':
        selected_method = change['new']
        # Show/hide relevant input boxes based on selection
        if selected_method == 'Username/Password':
            local_auth_box.layout.display = 'flex' # Show U/P box
            token_auth_box.layout.display = 'none'  # Hide Token box
        else: # Token (from ENV) method selected
            local_auth_box.layout.display = 'none'  # Hide U/P box
            token_auth_box.layout.display = 'none' # Hide Token box as input is not needed

        # Reset authentication status and token when method changes
        auth_status_label.value = 'Status: Not Authenticated (Method changed)'
        auth_status_label.style.text_color = None
        # Reset auth config state
        auth_config.current_token = None
        auth_config.current_user_info = None
        auth_config.api_client = None
        # Update global variables
        current_token = None
        current_user_info = None
        api_client = None

auth_method_selector.observe(on_auth_method_change, names='value')

In [10]:
# --- Authentication Button Functionality ---
def on_auth_button_clicked(b):
    global current_token, current_user_info, api_client # Allow modification of global state
    # Reset state before attempting authentication
    auth_config.current_token = None
    auth_config.current_user_info = None
    auth_config.api_client = None
    current_token = None
    current_user_info = None
    api_client = None

    auth_status_label.value = 'Status: Authenticating...'
    auth_status_label.style.text_color = 'orange'

    selected_name = oasis_dropdown.value
    base_url = oasis_options.get(selected_name)
    auth_method = auth_method_selector.value

    if not base_url:
        auth_status_label.value = 'Status: Error - Invalid Oasis selected.'
        auth_status_label.style.text_color = 'red'
        return

    try:
        if auth_method == 'Username/Password':
            # Get username and password from input widgets
            username = username_input.value.strip()
            password = password_input.value
            
            # If username or password is empty, try to get from environment
            env_username, env_password = get_credentials_from_env()
            # Use environment values to fill in any missing fields
            if not username and env_username:
                username = env_username

            if not password and env_password:
                password = env_password
                # Don't update password field for security
                password_input.placeholder = "Password loaded from ENV"

            # Check if we now have valid credentials
            if not username or not password:
                raise ValueError("Username and Password are required. They were not provided "
                               "in the form fields or found in environment variables.")

            # Use the auth module to get a token
            auth_config.current_token = get_token(base_url, username, password)
            # Clear password field after successful U/P auth for security
            password_input.value = ''
            # Reset placeholder to default if it was changed
            if password_input.placeholder == "Password loaded from ENV":
                password_input.placeholder = 'Enter Password or leave empty to use from ENV'

        elif auth_method == 'Token (from ENV)':
            # Read token from environment using the auth module
            auth_config.current_token = get_token_from_env()

        # Verify token and get user info using the auth module
        if auth_config.current_token:
            auth_config.current_user_info = verify_token(base_url, auth_config.current_token)
            # Create API client instance for future API calls
            auth_config.api_client = NomadClient(base_url, auth_config.current_token)

            # Update global variables for external use
            current_token = auth_config.current_token
            current_user_info = auth_config.current_user_info
            api_client = auth_config.api_client

            user_display = current_user_info.get('name', current_user_info.get('username', 'Unknown User'))
            auth_status_label.value = f'Status: Authenticated as {user_display} on {selected_name}.'
            auth_status_label.style.text_color = 'green'

        else:
             # This case should only be reached if something unexpected happened before token assignment
             raise ValueError("Authentication failed, no token available for verification.")

    except ValueError as ve: # Handle validation errors (missing input, missing ENV var)
         auth_status_label.value = f'Status: Error - {ve}'
         auth_status_label.style.text_color = 'red'
         # Reset auth config state and global variables on error
         auth_config.current_token = None
         auth_config.current_user_info = None
         auth_config.api_client = None
         current_token = None
         current_user_info = None
         api_client = None
    except ConnectionError as ce: # Handle network/HTTP errors
        auth_status_label.value = f'Status: {ce}'
        auth_status_label.style.text_color = 'red'
        # Reset auth config state and global variables on error
        auth_config.current_token = None
        auth_config.current_user_info = None
        auth_config.api_client = None
        current_token = None
        current_user_info = None
        api_client = None
    except Exception as e: # Catch any other unexpected errors
        auth_status_label.value = f'Status: Unexpected Error - {e}'
        auth_status_label.style.text_color = 'red'
        # Reset auth config state and global variables on error
        auth_config.current_token = None
        auth_config.current_user_info = None
        auth_config.api_client = None
        current_token = None
        current_user_info = None
        api_client = None

# Link the button's on_click event to the function
auth_button.on_click(on_auth_button_clicked)

In [11]:
# --- Initial Environment Detection ---
# Check if running in a known JupyterHub environment to set default auth method
is_hub_environment = bool(os.environ.get('JUPYTERHUB_USER'))

def initialize_auth_ui():
    """Initialize the authentication UI based on environment detection."""
    if is_hub_environment:
        # Default to Token method if in Hub, assuming token ENV var might be set
        auth_method_selector.value = 'Token (from ENV)'
        local_auth_box.layout.display = 'none'
        token_auth_box.layout.display = 'none' # Ensure token box is hidden
    else:
        # Default to Username/Password otherwise
        auth_method_selector.value = 'Username/Password'
        local_auth_box.layout.display = 'flex'
        token_auth_box.layout.display = 'none' # Ensure token box is hidden

initialize_auth_ui()

def get_auth_status():
    """Return the current authentication status as a dictionary."""
    return {
        'authenticated': bool(current_token and current_user_info and api_client),
        'token': current_token,
        'user_info': current_user_info,
        'client': api_client,
        'oasis': oasis_dropdown.value if oasis_dropdown.value else None,
        'oasis_url': oasis_options.get(oasis_dropdown.value, None) if oasis_dropdown.value else None
    }

def display_auth_ui():
    """Display the authentication UI."""
    display(general_settings_box)

# Note: The importing notebook must call display_auth_ui() to show the UI
# or simply use 'display(general_settings_box)' directly.

## Example Usage

Here's a simple example of how to use this authentication module in another notebook:

In [6]:
# This cell demonstrates how to use the auth module in another notebook
# Not meant to be executed in this notebook directly

'''
# In your dashboard notebook, run this to import the auth module
%run ./nomad_auth.ipynb

# Display the authentication UI
display_auth_ui()

# Later in your code, you can check if authentication was successful
if api_client:  # or use get_auth_status()['authenticated']
    print(f"Authenticated as {current_user_info.get('username')}")
    # Make API calls using api_client
    groups = api_client.get_groups()
    # ... rest of your dashboard code
else:
    print("Please authenticate first")
'''

'\n# In your dashboard notebook, run this to import the auth module\n%run ./nomad_auth.ipynb\n\n# Display the authentication UI\ndisplay_auth_ui()\n\n# Later in your code, you can check if authentication was successful\nif api_client:  # or use get_auth_status()[\'authenticated\']\n    print(f"Authenticated as {current_user_info.get(\'username\')}")\n    # Make API calls using api_client\n    groups = api_client.get_groups()\n    # ... rest of your dashboard code\nelse:\n    print("Please authenticate first")\n'